# Data Preparation using SAS Viya, CAS, and R

The focus for today's demonstration is using SAS Viya, CAS, and R to enable efficient data preparation, using multiple approaches:

- Use the *SWAT package* to make API calls to the Viya server and leverage the compute power of the CAS engine
- Can use standard R methods to reference and manipulate data, even when executing calculations on CAS
- Or can execute *CAS Actions* via SWAT calls
    - Today's example: the **dataSciencePilot action set**, which consists of actions that implement a policy-based, configurable, and scalable approach to automating data science workflows. This action set can be used to automate an end-to-end workflow or to automate steps in the  workflow such as data preparation, feature preprocessing, feature engineering, feature selection, and hyperparameter tuning.  More information about this action set is available on [its documentation page.](https://go.documentation.sas.com/?docsetId=casactml&docsetTarget=casactml_datasciencepilot_toc.htm&docsetVersion=8.5&locale=en)
***

## Table of Contents
Today we will set up the notebook, manage data, and apply eight actions available through the **dataSciencePilot** action set designed for data preparation.

1. [Setting Up the Notebook](#Setting-Up-the-Notebook)
1. [Managing Data](#Manage-Data)
1. [Prepare Data using dataSciencePilot action set](#Explore-Data)
    1. [Explore Correlations](#Explore-Correlations)
    1. [Analyze Missing Patterns](#Analyze-Missing-Patterns)
    1. [Detect Interactions](#Detect-Interactions)
    1. [Screen Variables](#Screen-Variables)
    1. [Feature Machine](#Feature-Machine)
    1. [Generate Shadow Features](#Generate-Shadow-Features)
    1. [Select Features](#Select-Features)
1. [Conclusion](#Conclusion)
***

## Setting Up the Notebook

First, we must load the Scripting Wrapper for Analytics Transfer (SWAT) package and use the package to connect to out Cloud Analytics Service (CAS).

In [1]:
library(swat)
library(stringr)

SWAT 1.4.1



In [2]:
server = CAS('localhost', 5570, authinfo='~/.authinfo', caslib="OpenDemo")

NOTE: Connecting to CAS and generating CAS action functions for loaded

      action sets...

NOTE: To generate the functions with signatures (for tab completion), set 

      options(cas.gen.function.sig=TRUE).



Now we will load action sets, which are analogous to packages in R (or libraries in Python).

In [3]:
loadActionSet(server, 'dataSciencePilot')
loadActionSet(server, 'table')

NOTE: Added action set 'dataSciencePilot'.

NOTE: Information for action set 'dataSciencePilot':

NOTE:    dataSciencePilot

NOTE:       exploreData - Exploration, automatic variable analysis and grouping using comprehensive statistical profiling of the variables.

NOTE:       screenVariables - Screens noise variables and variables that need special transformations to be useful in the downstream analytics.

NOTE:       analyzeMissingPatterns - Missing pattern analysis

NOTE:       exploreCorrelation - Explore linear and nonlinear correlation among the variables.

NOTE:       detectInteractions - Variable interaction detection and ranking

NOTE:       generateShadowFeatures - Generate shadow features.

NOTE:       featureMachine - Automated feature transformation and generation engine

NOTE:       selectFeatures - Feature selection

NOTE:       dsAutoMl - Automated machine learning pipeline exploration, execution and ranking.

NOTE: Added action set 'table'.

NOTE: Information for actio

## Manage Data

We can access and manage data that already exists on the Viya server. Here we will connect to a file previously saved to disk on the Viya server and "lift" that table in-memory in CAS. Throughout this example, we will use this data set for predicting home equity loan defaults. 

In [4]:
### If table is already loaded in-memory, first drop it before re-loading from disk:
hmeq_dropTbl <- cas.table.dropTable(server, caslib="OpenDemo", name="hmeq_from_R_Jupyter")


ERROR: The action stopped due to errors.



In [5]:
### Load table from disk:
hmeq_loadTbl <- cas.table.loadTable(server, caslib="Public", path="HMEQ.sashdat",
                                   casout=list(caslib="OpenDemo", name="hmeq_from_R_Jupyter"))

NOTE: Cloud Analytic Services made the file HMEQ.sashdat available as table HMEQ_FROM_R_JUPYTER in caslib OpenDemo.



We can now create a reference (called a 'CASTable') that points to that in-memory CAS table. This allows us to perform CAS Actions as well as standard interactions available on local R data.frames -- but all are executed on the server side in the CAS engine. 

In [6]:
hmeqCAStable <- defCasTable(server, tablename="hmeq_from_R_Jupyter", caslib="OpenDemo")

In [7]:
print("--- First six rows of the *in-memory* HMEQ table ---")
head(hmeqCAStable)

[1] "--- First six rows of the *in-memory* HMEQ table ---"


BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1100,25860,39025,HomeImp,Other,10.5,0,0,94.36667,1,9,NaN
1,1300,70053,68400,HomeImp,Other,7.0,0,2,121.83333,0,14,NaN
1,1500,13500,16700,HomeImp,Other,4.0,0,0,149.46667,1,10,NaN
1,1500,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1700,97800,112000,HomeImp,Office,3.0,0,0,93.33333,0,14,NaN
1,1700,30548,40320,HomeImp,Other,9.0,0,0,101.46600,1,8,37.11361


Our target is “BAD” meaning that it was a bad loan. I am setting up a variable to hold our target information as well as our policy information. Each policy is applicable to specific actions and I will provide more information about each policy later in the notebook. 

In [8]:
# Target Name 
trt <- "BAD"
# Exploration Policy 
# expo <- {'cardinality': {'lowMediumCutoff':40}}
expo <- list(cardinality = list(lowMediumCutoff=40))
# Screen Policy 
scpo <- list(missingPercentThreshold = 35)
# Selection Policy 
sepo <- list(criterion = "SU", topk = 4)
# Transformation Policy 
trpo <- list(entropy = TRUE, iqv = TRUE, kurtosis = TRUE, outlier = TRUE)

***
## Explore Data

The [exploreData action](https://go.documentation.sas.com/?docsetId=casactml&docsetTarget=casactml_datasciencepilot_details22.htm&docsetVersion=8.5&locale=en) calculates various statistical measures for each column in your data set such as Minimum, Maximum, Mean, Median, Mode, Number Missing, Standard Deviation, and more. The exploreData action also creates a hierarchical variable grouping with two levels. The first level groups variables according to their data type (interval, nominal, data, time, or datetime). The second level uses the following statistical metrics to group the interval and nominal data:
- Missing rate (interval and nominal).
- Cardinality (nominal). 
- Entropy (nominal). 
- Index of Qualitative Variation(IQV; interval and nominal). 
- Skewness (interval).
- Kurtosis (interval).
- Outliers (interval).
- Coefficient of Variation (CV; interval).

This action returns a CAS table listing all the variables, the variable groupings, and the summary statistics. These groupings allow for a pipelined approach to data transformation and cleaning. 

In [9]:
cas.dataSciencePilot.exploreData(
        server,
        table  = list(name ="hmeq_from_R_Jupyter", caslib="OpenDemo"),
        target = trt,     
        casOut = list(name = "EXPLORE_DATA_OUT_R", replace = TRUE),
        explorationPolicy = expo
    )

casLib,Name,Rows,Columns
<chr>,<chr>,<dbl>,<dbl>
CASUSER(sasdemo),EXPLORE_DATA_OUT_R,13,42


In [10]:
cas.table.fetch(server, table = list(name = "EXPLORE_DATA_OUT_R"))

_Index_,Variable,VarType,MissingRated,CardinalityRated,EntropyRated,IQVRated,CVRated,SkewnessRated,KurtosisRated,⋯,MomentCVPer,RobustCVPer,MomentSkewness,RobustSkewness,MomentKurtosis,RobustKurtosis,LowerOutlierMomentPer,UpperOutlierMomentPer,LowerOutlierRobustPer,UpperOutlierRobustPer
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,BAD,binary-target,NaN,NaN,NaN,NaN,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,REASON,character-nominal,1,1,3,NaN,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JOB,character-nominal,1,1,3,3,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LOAN,numeric-nominal,1,3,NaN,NaN,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MORTDUE,interval,2,NaN,NaN,NaN,3,1,2,⋯,60.27266,69.55352,1.8144807,0.8442205,6.4818663,0.37027422,0.0000000,2.9584712,2.2418229,1.72730614
6,VALUE,interval,1,NaN,NaN,NaN,3,1,3,⋯,56.38436,60.24788,3.0533443,0.9897549,24.3628049,0.42579341,0.0000000,2.4794802,0.4445964,2.59917921
7,YOJ,interval,2,NaN,NaN,NaN,3,1,1,⋯,84.88853,142.85714,0.9884601,0.9779435,0.3720725,-0.00610491,0.0000000,2.3140496,0.0000000,0.05509642
8,DEROG,numeric-nominal,2,1,2,1,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DELINQ,numeric-nominal,2,1,2,1,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


*** 
## Explore Correlations

If a target is specified, the [exploreCorrelation action](https://go.documentation.sas.com/?docsetId=casactml&docsetTarget=casactml_datasciencepilot_details21.htm&docsetVersion=8.5&locale=en) performs a linear and nonlinear correlation analysis of the input variables and the target. If a target is not specified, the exploreCorrelation action performs a linear and nonlinear correlation analysis for all pairwise combinations of the input variables. The correlation statistics available depend on the data type of each input variable in the pair. 
- Nominal-nominal correlation pairs have the following statistics available: Mutual Information (MI), Symmetric Uncertainty (SU), Information Value (IV; for binary target), Entropy, chi-square, G test (G2), and Cramer’s V. 
- Nominal-interval correlation pairs have the following statistics available: Mutual Information (MI), Symmetric Uncertainty (SU), Entropy, and F-test. 
- Interval-interval correlation pairs have the following statistics available: Mutual Information (MI), Symmetric Uncertainty (SU), Entropy, and Pearson correlation. 

This action returns a CAS table listing all the variable pairs and the correlation statistics. 

In [11]:
cas.dataSciencePilot.exploreCorrelation(
        server,
        table = list(name ="hmeq_from_R_Jupyter", caslib="OpenDemo"),
        casOut = list(name="CORR_from_R", replace=TRUE),
        target = trt
)
cas.table.fetch(server, table = list(name="CORR_from_R"))

casLib,Name,Rows,Columns
<chr>,<chr>,<dbl>,<dbl>
CASUSER(sasdemo),CORR_from_R,12,4


_Index_,FirstVariable,SecondVariable,Type,MI
<int>,<chr>,<chr>,<chr>,<dbl>
1,CLAGE,BAD,_it_,0.030242323
2,CLNO,BAD,_it_,0.015505245
3,DEBTINC,BAD,_it_,0.063484853
4,DELINQ,BAD,_it_,0.076942337
5,DEROG,BAD,_it_,0.048241053
6,LOAN,BAD,_it_,0.036786854
7,MORTDUE,BAD,_it_,0.012854994
8,NINQ,BAD,_it_,0.021362923
9,VALUE,BAD,_it_,0.016458330


***
## Analyze Missing Patterns

If the target is specified, the [analyzeMissingPatterns action](https://go.documentation.sas.com/?docsetId=casactml&docsetTarget=casactml_datasciencepilot_details04.htm&docsetVersion=8.5&locale=en) performs a missing pattern analysis of the input variables and the target. If a target is not specified, the analyzeMissingPatterns action performs a missing pattern analysis for all pairwise combinations of the input variables. This analysis provides the correlation strength between missing patterns across variable pairs and dependencies of missingness in one variable and the values of the other variable. This action returns a CAS table listing all the missing variable pairs and the statistics around missingness. 

In [12]:
cas.dataSciencePilot.analyzeMissingPatterns(
    server,
    table = list(name ="hmeq_from_R_Jupyter", caslib="OpenDemo"),
    target = trt, 
    casOut = list(name="MISS_PATTERN_from_R", replace=TRUE)
)


casLib,Name,Rows,Columns
<chr>,<chr>,<dbl>,<dbl>
CASUSER(sasdemo),MISS_PATTERN_from_R,12,7


In [13]:
cas.table.fetch(server, table = list(name="MISS_PATTERN_from_R"))

_Index_,FirstVariable,SecondVariable,Type,MI,NormMI,SU,EntropyPerChange
<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,CLAGE,BAD,_mt_,6.715406e-04,0.036635770,1.324012e-03,0.093150467
2,CLNO,BAD,_mt_,2.576016e-04,0.022695164,5.420620e-04,0.035732328
3,DEBTINC,BAD,_mt_,1.845951e-01,0.555613202,2.516096e-01,25.605476106
4,DELINQ,BAD,_mt_,3.061436e-03,0.078129153,5.182954e-03,0.424656620
5,DEROG,BAD,_mt_,3.953859e-03,0.088749839,6.342418e-03,0.548446081
6,LOAN,BAD,_mt_,0.000000e+00,0.000000000,0.000000e+00,0.000000000
7,MORTDUE,BAD,_mt_,1.127866e-05,0.004749429,1.966636e-05,0.001564481
8,NINQ,BAD,_mt_,1.243406e-03,0.049836964,2.176776e-03,0.172474876
9,VALUE,BAD,_mt_,3.591094e-02,0.263255279,8.395083e-02,4.981263843


***
## Detect Interactions

The [detectInteractions action](https://go.documentation.sas.com/?docsetId=casactml&docsetVersion=8.5&docsetTarget=casactml_datasciencepilot_details05.htm&locale=en) will assess the interactions between pairs of predictor variables and the correlation of that interaction on the response variable. Specially, it will see if the product of the pair of predictor variables correlate with the response variable. Since checking the correlation between the product of every predictor pair and the response variable can be computationally intensive, this action relies on the XYZ algorithm to search for these interactions efficiently in a high-dimensional space.   

The detectInteractions Action requires that all predictor variables be in a binary format, but the response variable can be numeric, binary, or multi-class.  Additionally, the detectInteractions Action can handle data in a sparse format, such as when predictor variables are encoded using an one-hot-encoding scheme.  In the example below, we will specify that our inputs are sparse. The output tables shows the gamma value for each pair of variables. 

In [14]:
# Tranform data for binary format
cas.dataPreprocess.transform(
    server,
    table = list(name ="hmeq_from_R_Jupyter", caslib="OpenDemo"),
    copyVars = "BAD", 
    casOut = list(name="hmeq_transform", replace=TRUE), 
    requestPackages = list(
        list(inputs=list("JOB", "REASON"),
            catTrans=list(
                method="label",
                arguments=list(
                    overrides=list(binMissing=TRUE)
                )
            )
        ),
        list(inputs=list("MORTDUE", "DEBTINC", "LOAN"),
             discretize=list(method="quantile", 
                             arguments=list(overrides=list(
                                 binMissing=TRUE)
                             )
             )
        )
    )
)

$TransInfo
  ActualName NTransVars           DisctMethod         CatTransMethod
1       _TR1          2                       Label (Sparse One-Hot)
2       _TR2          3 Equal-Freq (Quantile)                       

$VarTransInfo
  Variable Transformation    ResultVar    N NMiss NBins
1  DEBTINC           _TR2 _TR2_DEBTINC 4693  1267     6
2     LOAN           _TR2    _TR2_LOAN 5960     0     6
3  MORTDUE           _TR2 _TR2_MORTDUE 5442   518     6
4      JOB           _TR1     _TR1_JOB 5681   279   NaN
5   REASON           _TR1  _TR1_REASON 5708   252   NaN

$NomVarInfo
  Variable    N NMiss NLevels
1      JOB 5681   279       6
2   REASON 5708   252       2

$BinDetails
   Variable Transformation BinId  BinLowerBnd  BinUpperBnd     BinWidth NInBin
1   DEBTINC           _TR2     0          NaN          NaN          NaN   1267
2   DEBTINC           _TR2     1 5.244992e-01     27.61633 2.709183e+01    938
3   DEBTINC           _TR2     2 2.761633e+01     32.85570 5.239368e+00    939
4   DEBTINC           _TR2     3 3.285570e+01     36.58787 3.732168e+00    938
5   DEBTINC           _TR2     4 3.658787e+01     39.85215 3.264277e+00    939
6   DEBTINC           _TR2     5 3.985215e+01    203.31215 1.634600e+02    939
7      LOAN           _TR2     0          NaN          NaN          NaN      0
8      LOAN           _TR2     1 1.100000e+03  10000.00000 8.900000e+03   1130
9      LOAN           _TR2     2 1.000000e+04  14400.00000 4.400000e+03   1253
10     LOAN           _TR2     3 1.440000e+04  18800.00000 4.400000e+03   1187
11     LOAN           _TR2     4 1.880000e+04  25000.00000 6.200000e+03   1146
12     LOAN           _TR2     5 2.500000e+04  89900.00000 6.490000e+04   1244
13  MORTDUE           _TR2     0          NaN          NaN          NaN    518
14  MORTDUE           _TR2     1 2.063000e+03  41656.00000 3.959300e+04   1088
15  MORTDUE           _TR2     2 4.165600e+04  57816.00000 1.616000e+04   1088
16  MORTDUE           _TR2     3 5.781600e+04  74000.00000 1.618400e+04   1088
17  MORTDUE           _TR2     4 7.400000e+04  99300.00000 2.530000e+04   1089
18  MORTDUE           _TR2     5 9.930000e+04 399550.00000 3.002500e+05   1089
           Mean          Std          Min          Max
1           NaN          NaN          NaN          NaN
2      22.34201 5.042519e+00 5.244992e-01     27.61084
3      30.30564 1.488466e+00 2.761633e+01     32.85379
4      34.82965 1.061672e+00 3.285570e+01     36.57539
5      38.22702 9.338563e-01 3.658787e+01     39.84839
6      43.18420 9.388867e+00 3.985215e+01    203.31215
7           NaN          NaN          NaN          NaN
8    7112.12389 1.964838e+03 1.100000e+03   9900.00000
9   12049.08220 1.278119e+03 1.000000e+04  14300.00000
10  16412.38416 1.245880e+03 1.440000e+04  18700.00000
11  21673.21117 1.690763e+03 1.880000e+04  24900.00000
12  34927.89389 1.223016e+04 2.500000e+04  89900.00000
13          NaN          NaN          NaN          NaN
14  25995.50244 1.013401e+04 2.063000e+03  41641.00000
15  50025.34877 4.514639e+03 4.165600e+04  57800.00000
16  65523.41905 4.756172e+03 5.781600e+04  73996.00000
17  85389.87374 7.287845e+03 7.400000e+04  99204.00000
18 141796.72053 4.476920e+04 9.930000e+04 399550.00000

$OutputCasTables
            casLib           Name Rows Columns
1 CASUSER(sasdemo) hmeq_transform 5960       6

In [15]:
cas.table.fetch(server, table = list(name = "hmeq_transform"))

_Index_,BAD,_TR2_DEBTINC,_TR2_LOAN,_TR2_MORTDUE,_TR1_JOB,_TR1_REASON
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,1,1,3,2
2,1,0,1,3,3,2
3,1,0,1,1,3,2
4,1,0,1,0,0,0
5,0,0,1,4,2,2
6,1,4,1,1,3,2
7,1,0,1,2,3,2
8,1,4,1,1,3,2
9,1,0,1,1,3,2


In [16]:
cas.dataSciencePilot.detectInteractions(
    server,
    table ='hmeq_transform', 
    target = trt, 
    event = '1', 
    sparse = TRUE, 
    inputs = list("_TR1_JOB", "_TR1_REASON", "_TR2_MORTDUE", "_TR2_DEBTINC", "_TR2_LOAN"), 
    inputLevels = list(7, 3, 6, 6, 6), 
    casOut = list(name = "DETECT_INT_OUT_from_R", replace=TRUE)
)

$AnalysisInfo
                              Description        Value
1             Number of Observations Read 5960.0000000
2             Number of Observations Used 4039.0000000
3                        Number of Inputs    5.0000000
4 Background Average Interaction Strength    0.3214374
5                          Subsample Size    3.0000000

$OutputCasTables
            casLib                  Name Rows Columns
1 CASUSER(sasdemo) DETECT_INT_OUT_from_R  135       5

***
## Screen Variables

The [screenVariables action](https://go.documentation.sas.com/?docsetId=casactml&docsetTarget=casactml_datasciencepilot_details25.htm&docsetVersion=8.5&locale=en) makes one of the following recommendations for each input variable:
-	Remove variable if there are significant data-quality issues. 
-	Transform and keep variable if there are some data-quality issues. 
-	Keep variable if there are no data quality issues. 

The screenVariables action considers the following features of the input variables to make its recommendation:
-	Missing rate exceeds  threshold in screenPolicy (default is 90). 
-	Constant value across input variable.  
-	Mutual Information (MI) about the target is below the threshold in the screenPolicy (default is 0.05)
-	Entropy across levels. 
-	Entropy reduction of target exceeds threshold in screenPolicy (default is 90); also referred to as leakage. 
-	Symmetric Uncertainty (SU) of two variables exceed threshold in screenPolicy (default is 1); also referred to as redundancy. 

This action returns a CAS table listing all the input variables, the recommended action, and the reason for the recommended action.  

In [17]:
cas.dataSciencePilot.screenVariables(
    server,
    table = list(name ="hmeq_from_R_Jupyter", caslib="OpenDemo"),
    target = trt, 
    casOut = list(name="SCREEN_VARIABLES_FROM_R", replace=TRUE)
)

cas.table.fetch(server, table = list(name="SCREEN_VARIABLES_FROM_R"))

casLib,Name,Rows,Columns
<chr>,<chr>,<dbl>,<dbl>
CASUSER(sasdemo),SCREEN_VARIABLES_FROM_R,12,3


_Index_,Variable,Recommendation,Reason
<int>,<chr>,<chr>,<chr>
1,REASON,keep,passed all screening tests
2,JOB,keep,passed all screening tests
3,LOAN,keep,passed all screening tests
4,MORTDUE,keep,passed all screening tests
5,VALUE,keep,passed all screening tests
6,YOJ,keep,passed all screening tests
7,DEROG,keep,passed all screening tests
8,DELINQ,keep,passed all screening tests
9,CLAGE,keep,passed all screening tests


*** 
## Feature Machine

The [featureMachine action](https://go.documentation.sas.com/?docsetId=casactml&docsetTarget=casactml_datasciencepilot_details23.htm&docsetVersion=8.5&locale=en) creates an automated and parallel generation of features. The featureMachine action first explores the data and groups the input variables into categories with the same statistical profile, like the exploreData action. Next the featureMachine action screens variables to identify noise variables to exclude from further analysis, like the screenVariables action.  Finally, the featureMachine action generates new features by using the available structured pipelines:
-	Missing indicator addition. 
-	Mode imputation and rare value grouping. 
-	Missing level and rare value grouping. 
-	Median imputation. 
-	Mode imputation and label encoding. 
-	Missing level and label encoding. 
-	Yeo-Johnson transformation and median imputation. 
-	Box-Cox transformation. 
-	Quantile binning with missing bins.
-	Regression tree binning.
-	Decision tree binning. 
-	MDLP binning. 
-	Target encoding. 
-	Date, time, and datetime transformations. 

Depending on the parameters specified in the transformationPolicy, the featureMachine action can generate several features for each input variable. This action returns four CAS tables: the first lists information around the transformation pipelines, the second lists information around the transformed features, the third is the input table scored with the transformed features, and the fourth is an analytical store for scoring any additional input tables. 

In [18]:
cas.dataSciencePilot.featureMachine(
    server,
    table = list(name ="hmeq_from_R_Jupyter", caslib="OpenDemo"),
    target = trt, 
    copyVars = trt, 
    explorationPolicy = expo, 
    screenPolicy = scpo, 
    transformationPolicy = trpo, 
    transformationOut       = list(name="TRANSFORMATION_OUT", replace=TRUE),
    featureOut              = list(name="FEATURE_OUT", replace=TRUE),
    casOut                  = list(name="CAS_OUT", replace=TRUE),
    saveState               = list(name="ASTORE_OUT", replace=TRUE)
)

casLib,Name,Rows,Columns
<chr>,<chr>,<dbl>,<dbl>
CASUSER(sasdemo),TRANSFORMATION_OUT,33,21
CASUSER(sasdemo),FEATURE_OUT,59,9
CASUSER(sasdemo),CAS_OUT,5960,60
CASUSER(sasdemo),ASTORE_OUT,1,2


In [19]:
cas.table.fetch(server, table = list(name="TRANSFORMATION_OUT"))

_Index_,FTGPipelineId,Name,NVariables,IsInteraction,ImputeMethod,OutlierMethod,OutlierTreat,OutlierArgs,FunctionMethod,⋯,MapIntervalArgs,HashMethod,HashArgs,DateTimeMethod,DiscretizeMethod,DiscretizeArgs,CatTransMethod,CatTransArgs,InteractionMethod,InteractionSynthesizer
<int>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,⋯,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,1,miss_ind,5,,,,,NaN,,⋯,NaN,MissIndicator,2,,,NaN,,NaN,,
2,2,grp_rare1,2,,Mode,,,NaN,,⋯,NaN,,NaN,,,NaN,Group Rare,5,,
3,3,hc_tar_frq_rat,1,,,,,NaN,,⋯,10,,NaN,,,NaN,,NaN,,
4,4,hc_lbl_cnt,1,,,,,NaN,,⋯,0,,NaN,,,NaN,,NaN,,
5,5,hc_cnt,1,,,,,NaN,,⋯,0,,NaN,,,NaN,,NaN,,
6,6,hc_cnt_log,1,,,,,NaN,Log,⋯,0,,NaN,,,NaN,,NaN,,
7,7,lchehi_lab,1,,,,,NaN,,⋯,NaN,,NaN,,,NaN,Label (Sparse One-Hot),0,,
8,8,lcnhenhi_grp_rare,1,,,,,NaN,,⋯,NaN,,NaN,,,NaN,Group Rare,5,,
9,9,lcnhenhi_dtree5,1,,,,,NaN,,⋯,NaN,,NaN,,,NaN,DTree,5,,


In [20]:
cas.table.fetch(server, table = list(name="FEATURE_OUT"))

_Index_,FeatureId,Name,IsNominal,FTGPipelineId,NInputs,InputVar1,InputVar2,InputVar3,Label
<int>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,1,cpy_int_med_imp_CLAGE,0,32,1,CLAGE,,,CLAGE: Low missing rate - median imputation
2,2,miss_ind_CLAGE,1,1,1,CLAGE,,,CLAGE: Significant missing - missing indicator
3,3,nhoks_nloks_dtree_10_CLAGE,1,31,1,CLAGE,,,"CLAGE: Not high (outlier, kurtosis, skewness) - ten bin decision tree binning"
4,4,nhoks_nloks_dtree_5_CLAGE,1,30,1,CLAGE,,,"CLAGE: Not high (outlier, kurtosis, skewness) - five bin decision tree binning"
5,5,nhoks_nloks_log_CLAGE,0,26,1,CLAGE,,,"CLAGE: Not high (outlier, kurtosis, skewness) - log + impute(median)"
6,6,nhoks_nloks_pow_n0_5_CLAGE,0,25,1,CLAGE,,,"CLAGE: Not high (outlier, kurtosis, skewness) - power(-0.5) + impute(median)"
7,7,nhoks_nloks_pow_n1_CLAGE,0,24,1,CLAGE,,,"CLAGE: Not high (outlier, kurtosis, skewness) - power(-1) + impute(median)"
8,8,nhoks_nloks_pow_n2_CLAGE,0,23,1,CLAGE,,,"CLAGE: Not high (outlier, kurtosis, skewness) - power(-2) + impute(median)"
9,9,nhoks_nloks_pow_p0_5_CLAGE,0,27,1,CLAGE,,,"CLAGE: Not high (outlier, kurtosis, skewness) - power(0.5) + impute(median)"


In [21]:
cas.table.fetch(server, table = list(name="CAS_OUT"))

_Index_,BAD,cpy_int_med_imp_CLAGE,miss_ind_CLAGE,nhoks_nloks_dtree_10_CLAGE,nhoks_nloks_dtree_5_CLAGE,nhoks_nloks_log_CLAGE,nhoks_nloks_pow_n0_5_CLAGE,nhoks_nloks_pow_n1_CLAGE,nhoks_nloks_pow_n2_CLAGE,⋯,hc_lbl_cnt_LOAN,hc_tar_frq_rat_LOAN,cpy_nom_miss_lev_lab_NINQ,lcnhenhi_dtree10_NINQ,lcnhenhi_dtree5_NINQ,lcnhenhi_grp_rare_NINQ,miss_ind_NINQ,cpy_nom_miss_lev_lab_JOB,lchehi_lab_JOB,cpy_nom_miss_lev_lab_REASON
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,94.36667,1,3,2,4.557729,0.10240041,0.010485844,1.099529e-04,⋯,528,0.5,2,2,2,2,1,3,3,2
2,1,121.83333,1,4,2,4.810828,0.09022811,0.008141113,6.627771e-05,⋯,461,0.5,1,1,1,1,1,3,3,2
3,1,149.46667,1,4,2,5.013742,0.08152294,0.006645990,4.416919e-05,⋯,385,0.5,2,2,2,2,1,3,3,2
4,1,173.46667,0,0,0,5.161734,0.07570835,0.005731754,3.285300e-05,⋯,385,0.5,0,0,0,0,0,0,0,0
5,0,93.33333,1,2,2,4.546835,0.10295973,0.010600707,1.123750e-04,⋯,359,0.5,1,1,1,1,1,2,2,2
6,1,101.46600,1,3,2,4.629531,0.09878934,0.009759335,9.524461e-05,⋯,359,0.5,2,2,2,2,1,3,3,2
7,1,77.10000,1,2,2,4.357990,0.11315519,0.012804097,1.639449e-04,⋯,401,0.5,2,2,2,2,1,3,3,2
8,1,88.76603,1,2,2,4.497207,0.10554654,0.011140072,1.241012e-04,⋯,401,0.5,1,1,1,1,1,3,3,2
9,1,216.93333,1,7,4,5.384189,0.06773890,0.004588559,2.105488e-05,⋯,259,0.5,2,2,2,2,1,3,3,2


*** 
## Generate Shadow Features

The [generateShadowFeatures Action](https://go.documentation.sas.com/?docsetId=casactml&docsetTarget=casactml_datasciencepilot_examples39.htm&docsetVersion=8.5&locale=en) performs a scalable random permutation of input features to create shadow features. The shadow features are randomly selected from a matching distribution of each input feature. These shadow features can be used for all-relevant feature selection which removes the inputs whose variable importance is lower than the shadow feature’s variable importance. The shadow features can also be used in a post-fit analysis using Permutation Feature Importance (PFI). By replacing each input with its shadow feature one-by-one and measuring the change on model performance, one can determine that features importance based on relative size of the model’s performance change.  

In the example below, I will use the outputs of the feature machine for all-relevant feature selection. This involves getting the variable metadata from my feature machine table, generating my shadow features, finding the variable importance for my features and shadow features using a random forest, and comparing each variable's performance to its shadow features. In the end, I will only keep variables with a higher importance than its shadow feature for the next phase. 

In [22]:
# Getting variable names and metadata from feature machine output
fm <- to.casDataFrame(defCasTable(server, "FEATURE_OUT"))
inputs <- fm$Name
nom <- fm$Name[fm$IsNominal==1]

# Generating Shadow Features
cas.dataSciencePilot.generateShadowFeatures(
    server,
    table = 'CAS_OUT', 
    nProbes = 2, 
    inputs = inputs, 
    nominals = nom,
    casout=list(name = "SHADOW_FEATURES_OUT", replace=TRUE),
    copyVars = trt
)


casLib,Name,Rows,Columns
<chr>,<chr>,<dbl>,<dbl>
CASUSER(sasdemo),SHADOW_FEATURES_OUT,5960,119


In [23]:
cas.table.fetch(server, table = "SHADOW_FEATURES_OUT")

_Index_,BAD,_fpi_cpy_int_med_imp_CLAGE_1,_fpi_cpy_int_med_imp_CLAGE_2,_fpi_cpy_int_med_imp_DEBTINC_1,_fpi_cpy_int_med_imp_DEBTINC_2,_fpi_cpy_int_med_imp_MORTDUE_1,_fpi_cpy_int_med_imp_MORTDUE_2,_fpi_cpy_int_med_imp_VALUE_1,_fpi_cpy_int_med_imp_VALUE_2,⋯,_fpn_miss_ind_YOJ_1,_fpn_miss_ind_YOJ_2,_fpn_nhoks_nloks_dtree_10_CLAGE_1,_fpn_nhoks_nloks_dtree_10_CLAGE_2,_fpn_nhoks_nloks_dtree_10_YOJ_1,_fpn_nhoks_nloks_dtree_10_YOJ_2,_fpn_nhoks_nloks_dtree_5_CLAGE_1,_fpn_nhoks_nloks_dtree_5_CLAGE_2,_fpn_nhoks_nloks_dtree_5_YOJ_1,_fpn_nhoks_nloks_dtree_5_YOJ_2
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,367.24187,215.61179,22.05703,34.81827,76600.853,54077.154,68489.29,54514.17,⋯,1,0,2,6,8,8,1,5,4,4
2,1,102.51186,143.73687,30.70182,42.49770,22112.279,33337.008,62824.75,131523.25,⋯,1,1,5,10,2,3,2,5,0,2
3,1,206.62355,150.15827,40.00731,34.81826,75961.838,16355.440,96608.76,50000.39,⋯,1,1,10,10,1,9,3,2,4,0
4,1,295.08331,171.72670,40.95501,34.81826,131443.202,43809.377,96268.25,67192.53,⋯,1,1,1,2,2,4,0,2,5,4
5,0,174.44322,97.40284,39.93486,31.55124,62212.928,43778.751,84083.80,49013.17,⋯,1,1,1,3,10,1,2,5,4,3
6,1,130.82698,261.91422,39.42332,34.81826,56805.911,137001.396,171706.44,184217.50,⋯,1,1,4,10,5,7,2,2,2,4
7,1,223.86455,267.67097,40.41403,43.24281,51989.981,125690.917,45654.14,131367.07,⋯,1,1,4,9,9,8,0,5,2,4
8,1,174.55157,273.31307,19.24355,43.78095,59903.271,56715.712,35400.55,108632.44,⋯,1,1,9,9,3,10,1,0,4,5
9,1,203.03879,108.60735,30.44789,38.72168,50371.576,41130.316,48047.23,61456.22,⋯,1,1,4,9,0,7,2,5,2,1


In [24]:
# First, need to load the decisionTree Action Set:
loadActionSet(server, 'decisionTree')

# Getting Feature Importance for Original Features
feats <- cas.decisionTree.forestTrain(
    server,
    table = 'CAS_OUT', 
    inputs = inputs, 
    target = trt, 
    varImp = TRUE)

real_features <- feats$DTreeVarImpInfo


# Getting Feature Importance for Shadow Features
inp <- colnames(defCasTable(server, 'SHADOW_FEATURES_OUT'))
shadow_feats <- cas.decisionTree.forestTrain(
    server,
    table = 'SHADOW_FEATURES_OUT', 
    inputs = inp, 
    target = trt, 
    varImp = TRUE)

sf <- shadow_feats$DTreeVarImpInfo


# Building dataframe for easy comparison
feat_comp <- data.frame(Variable=real_features$Variable, 
                        Real_Imp=real_features$Importance,
                        SF_Imp1=0,
                        SF_Imp2=0) 

NOTE: Added action set 'decisionTree'.

NOTE: Information for action set 'decisionTree':

NOTE:    decisionTree

NOTE:       dtreeTrain - Trains a decision tree

NOTE:       dtreeScore - Scores a table using a decision tree model

NOTE:       dtreeSplit - Splits decision tree nodes

NOTE:       dtreePrune - Prune a decision tree

NOTE:       dtreeMerge - Merges decision tree nodes

NOTE:       dtreeCode - Generates DATA step scoring code from a decision tree model

NOTE:       forestTrain - Trains a forest. This action requires a SAS Visual Data Mining and Machine Learning license

NOTE:       forestScore - Scores a table using a forest model

NOTE:       forestCode - Generates DATA step scoring code from a forest model

NOTE:       gbtreeTrain - Trains a gradient boosting tree. This action requires a SAS Visual Data Mining and Machine Learning license

NOTE:       gbtreeScore - Scores a table using a gradient boosting tree model

NOTE:       gbtreeCode - Generates DATA step scoring co

In [25]:
### Finding each Feature's Shadow Feature
  ### loop through shadow features data object:
    for (cRow in 1:nrow(sf) ) {
        ### remove the first five characters ("_fpn_") and final two characters ("_N")
        temp_name <- str_sub(sf$Variable[cRow], 6, -3)
        ### grab _N final numeric indicator
        temp_num <- str_sub(sf$Variable[cRow], -1, -1)
        ### then loop through each row of feat_comp (feature comparison object) and fill in importance values:
        for (fcRow in 1:nrow(feat_comp) ) {
            if (temp_name == feat_comp$Variable[fcRow]) {
                if (temp_num == 1) {
                    ### fill in first shadow feature's importance value:
                    feat_comp$SF_Imp1[fcRow] <- sf$Importance[cRow]
                }
                else {
                    ### fill in second shadow feature's importance value:
                    feat_comp$SF_Imp2[fcRow] <- sf$Importance[cRow]
                }
            }
        }
    }

In [26]:
# Determining which features have an importance smaller than their shadow feature's importance
feat_comp$to_drop <- ifelse(feat_comp$Real_Imp <= feat_comp$SF_Imp1, 1, 
                            ifelse(feat_comp$Real_Imp <= feat_comp$SF_Imp2, 1, 0))

cols_to_drop <- as.character(feat_comp$Variable[feat_comp$to_drop==1])
"Dropping the following columns: "
print(paste(cols_to_drop))

[1] "Dropping the following columns: "

 [1] "hc_cnt_log_LOAN"          "nhoks_nloks_pow_p0_5_YOJ"
 [3] "nhoks_nloks_pow_n0_5_YOJ" "nhoks_nloks_pow_p2_YOJ"  
 [5] "ho_quan_disct10_MORTDUE"  "hc_cnt_LOAN"             
 [7] "nhoks_nloks_pow_n1_YOJ"   "ho_winsor_VALUE"         
 [9] "ho_winsor_MORTDUE"        "hc_lbl_cnt_LOAN"         
[11] "nhoks_nloks_pow_n2_YOJ"   "nhoks_nloks_pow_p1_YOJ"  
[13] "nhoks_nloks_dtree_5_YOJ" 


In [27]:
### drop the columns using the alterTable CAS Action:
cas.table.alterTable(server, name = "CAS_OUT", drop = cols_to_drop)

list()

*** 
## Select Features

The [selectFeatures action](https://go.documentation.sas.com/?docsetId=casactml&docsetTarget=casactml_datasciencepilot_details26.htm&docsetVersion=8.5&locale=en) performs a filter-based selection by the criterion selected in the selectionPolicy (default is the best ten input variables according to the Mutual Information statistic). The criterion available for selection include Chi-Square, Cramer’s V, F-test, G2, Information Value, Mutual Information, Normalized Mutual Information statistic, Pearson correlation, and the Symmetric Uncertainty statistic. This action returns a CAS table listing the variables, their rank according to the selected criterion, and the value of the selected criterion. 

In [28]:
cas.dataSciencePilot.screenVariables(
    server,
    table='CAS_OUT', 
    target=trt, 
    screenPolicy=scpo, 
    casout=list(name="SCREEN_VARIABLES_OUT", replace=TRUE)
)

cas.table.fetch(server, table = "SCREEN_VARIABLES_OUT")

casLib,Name,Rows,Columns
<chr>,<chr>,<dbl>,<dbl>
CASUSER(sasdemo),SCREEN_VARIABLES_OUT,46,3


_Index_,Variable,Recommendation,Reason
<int>,<chr>,<chr>,<chr>
1,cpy_int_med_imp_CLAGE,keep,passed all screening tests
2,miss_ind_CLAGE,keep,passed all screening tests
3,nhoks_nloks_dtree_10_CLAGE,keep,passed all screening tests
4,nhoks_nloks_dtree_5_CLAGE,keep,passed all screening tests
5,nhoks_nloks_log_CLAGE,keep,passed all screening tests
6,nhoks_nloks_pow_n0_5_CLAGE,keep,passed all screening tests
7,nhoks_nloks_pow_n1_CLAGE,keep,passed all screening tests
8,nhoks_nloks_pow_n2_CLAGE,keep,passed all screening tests
9,nhoks_nloks_pow_p0_5_CLAGE,keep,passed all screening tests


***
## Data Science Automated Machine Learning Pipeline

The [dsAutoMl action](https://go.documentation.sas.com/?docsetId=casactml&docsetTarget=casactml_datasciencepilot_details20.htm&docsetVersion=8.5&locale=en) creates a policy-based, scalable, end-to-end automated machine learning pipeline for both regression and classification problems. The only input required from the user is the input data set and the target variable, but optional parameters include the policy parameters for data exploration, variable screening, feature selection, and feature transformation.  Overriding the default policy parameters allow a data scientist to configure their pipeline in their data science workflow. In addition, a data scientist may also select additional models to consider. By default, only a decision tree model is included in the pipeline, but neural networks, random forest models, and gradient boosting models are also available. 

The dsAutoMl action first explores the data and groups the input variables into categories with the same statistical profile, like the exploreData action. Next the dsAutoMl action screens variables to identify noise variables to exclude from further analysis, like the screenVariables action.  Then, the dsAutoMl action generates several new features for the input variables, like the featureMachine action. After there are various new cleaned features, the dsAutoMl action will select features based on selected criterion, like the selectFeatures action. 

From here, various pipelines are created using subsets of the selected features, chosen for each pipeline using a feature-representation algorithm. Then the chosen models are added to each pipeline and the hyperparameters for the selected models are optimized, like the modelComposer action of the Autotune action set. These hyperparameters are optimized for the selected objective parameter when cross-validated. By default, classification problems are optimized to have the smallest Misclassification Error Rate (MCE) and regression problems are optimized to have the smallest Average Square Error (ASR).  Data scientists can then select their champion and challenger models from the pipelines. 

This action returns several CAS tables: the first lists information around the transformation pipelines, the second lists information around the transformed features, the third lists pipeline performance according to the objective parameter and the last tables are analytical stores for creating the feature set and scoring  with our model when new data is available.

In [29]:
cas.dataSciencePilot.dsAutoMl(
    server,
    table = list(name ="hmeq_from_R_Jupyter", caslib="OpenDemo"),
    target = trt, 
    explorationPolicy = expo, 
    screenPolicy = scpo, 
    selectionPolicy = sepo,
    transformationPolicy = trpo,
     modelTypes              = c("decisionTree", "gradboost"),
        objective               = "ASE",
        sampleSize              = 10,
        topKPipelines           = 10,
        kFolds                  = 5,
        transformationOut       = list(name="TRANSFORMATION_OUT_R", replace=TRUE),
        featureOut              = list(name="FEATURE_OUT_R", replace=TRUE),
        pipelineOut             = list(name="PIPELINE_OUT_R", replace=TRUE),
        saveState               = list(modelNamePrefix="ASTORE_OUT_R", replace=TRUE)
)

NOTE: Added action set 'autotune'.


NOTE: Added action set 'decisionTree'.




NOTE: Early stopping is activated; 'NTREE' will not be tuned.

NOTE: Added action set 'autotune'.


NOTE: The number of bins will not be tuned since all inputs are nominal.

NOTE: Added action set 'decisionTree'.




NOTE: Early stopping is activated; 'NTREE' will not be tuned.

NOTE: The number of bins will not be tuned since all inputs are nominal.

NOTE: Added action set 'autotune'.


NOTE: Added action set 'decisionTree'.




NOTE: Early stopping is activated; 'NTREE' will not be tuned.

NOTE: Added action set 'autotune'.


NOTE: The number of bins will not be tuned since all inputs are nominal.

NOTE: Added action set 'decisionTree'.




NOTE: Early stopping is activated; 'NTREE' will not be tuned.

NOTE: The number of bins will not be tuned since all inputs are nominal.

NOTE: Added action set 'autotune'.


NOTE: The number of bins will not be tuned since all inputs are nominal.

NOTE: Added action se

Descr,Value
<chr>,<dbl>
Number of Tree Nodes,15.00000
Max Number of Branches,2.00000
Number of Levels,6.00000
Number of Leaves,8.00000
Number of Bins,50.00000
Minimum Size of Leaves,38.00000
Maximum Size of Leaves,4179.00000
Number of Variables,1.00000
Confidence Level for Pruning,0.25000


In [30]:
cas.table.fetch(server, table = list(name = "TRANSFORMATION_OUT_R"))

_Index_,FTGPipelineId,Name,NVariables,IsInteraction,ImputeMethod,OutlierMethod,OutlierTreat,OutlierArgs,FunctionMethod,⋯,MapIntervalArgs,HashMethod,HashArgs,DateTimeMethod,DiscretizeMethod,DiscretizeArgs,CatTransMethod,CatTransArgs,InteractionMethod,InteractionSynthesizer
<int>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,⋯,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,1,miss_ind,3,,,,,NaN,,⋯,NaN,MissIndicator,2,,,NaN,Label (Sparse One-Hot),NaN,,
2,2,hc_tar_frq_rat,1,,,,,NaN,,⋯,10,,NaN,,,NaN,,NaN,,
3,3,hc_lbl_cnt,1,,,,,NaN,,⋯,0,,NaN,,,NaN,,NaN,,
4,4,hc_cnt,1,,,,,NaN,,⋯,0,,NaN,,,NaN,,NaN,,
5,5,hc_cnt_log,1,,,,,NaN,Log,⋯,0,,NaN,,,NaN,,NaN,,
6,6,lcnhenhi_grp_rare,2,,,,,NaN,,⋯,NaN,,NaN,,,NaN,Group Rare,5,,
7,7,lcnhenhi_dtree5,2,,,,,NaN,,⋯,NaN,,NaN,,,NaN,DTree,5,,
8,8,lcnhenhi_dtree10,2,,,,,NaN,,⋯,NaN,,NaN,,,NaN,DTree,10,,
9,9,hk_yj_n2,1,,Median,,,NaN,Yeo-Johnson,⋯,NaN,,NaN,,,NaN,,NaN,,


In [31]:
cas.table.fetch(server, table = list(name = "FEATURE_OUT_R"))

_Index_,FeatureId,Name,IsNominal,FTGPipelineId,NInputs,InputVar1,InputVar2,InputVar3,Label,RankCrit,BestTransRank,GlobalIntervalRank,GlobalNominalRank,GlobalRank,IsGenerated
<int>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,cpy_int_med_imp_DEBTINC,0,16,1,DEBTINC,,,DEBTINC: Low missing rate - median imputation,0.086482795,1,1,NaN,4,1
2,2,hk_dtree_disct10_DEBTINC,1,15,1,DEBTINC,,,DEBTINC: High kurtosis - ten bin decision tree binning,0.102374280,3,NaN,3,3,0
3,3,hk_dtree_disct5_DEBTINC,1,14,1,DEBTINC,,,DEBTINC: High kurtosis - five bin decision tree binning,0.129960492,2,NaN,2,2,1
4,4,hk_yj_0_DEBTINC,0,11,1,DEBTINC,,,DEBTINC: High kurtosis - Yeo-Johnson(lambda=0) + impute(median),0.080955282,3,3,NaN,6,0
5,5,hk_yj_n1_DEBTINC,0,10,1,DEBTINC,,,DEBTINC: High kurtosis - Yeo-Johnson(lambda=-1) + impute(median),0.060570668,4,4,NaN,9,0
6,6,hk_yj_n2_DEBTINC,0,9,1,DEBTINC,,,DEBTINC: High kurtosis - Yeo-Johnson(lambda=-2) + impute(median),0.007162127,6,10,NaN,17,0
7,7,hk_yj_p1_DEBTINC,0,12,1,DEBTINC,,,DEBTINC: High kurtosis - Yeo-Johnson(lambda=1) + impute(median),0.086482795,1,1,NaN,4,1
8,8,hk_yj_p2_DEBTINC,0,13,1,DEBTINC,,,DEBTINC: High kurtosis - Yeo-Johnson(lambda=2) + impute(median),0.044039344,5,5,NaN,12,0
9,9,miss_ind_DEBTINC,1,1,1,DEBTINC,,,DEBTINC: Significant missing - missing indicator,0.251609647,1,NaN,1,1,1


In [32]:
cas.table.fetch(server, table = list(name = "PIPELINE_OUT_R"))

_Index_,PipelineId,ModelType,MLType,Objective,ObjectiveType,Target,NFeatures,Feat1Id,Feat1IsNom,Feat2Id,Feat2IsNom,Feat3Id,Feat3IsNom,Feat4Id,Feat4IsNom
<int>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,binary classification,dtree,0.1194631,MCE,BAD,4,10,1,15,1,9,1,23,0
2,6,binary classification,gradBoost,0.1202823,MCE,BAD,4,13,1,18,1,3,1,23,0
3,2,binary classification,gradBoost,0.1241641,MCE,BAD,4,10,1,15,1,9,1,23,0
4,5,binary classification,dtree,0.1275376,MCE,BAD,4,13,1,18,1,3,1,23,0
5,7,binary classification,dtree,0.1803706,MCE,BAD,1,10,1,NaN,NaN,NaN,NaN,NaN,NaN
6,9,binary classification,dtree,0.1825513,MCE,BAD,1,13,1,NaN,NaN,NaN,NaN,NaN,NaN
7,8,binary classification,gradBoost,0.1852033,MCE,BAD,1,10,1,NaN,NaN,NaN,NaN,NaN,NaN
8,3,binary classification,dtree,0.1862319,MCE,BAD,1,13,1,NaN,NaN,NaN,NaN,NaN,NaN
9,4,binary classification,gradBoost,0.1993287,MCE,BAD,1,13,1,NaN,NaN,NaN,NaN,NaN,NaN


***
## Conclusion

The dataSciencePilot action set consists of actions that implement a policy-based, configurable, and scalable approach to automating data science workflows. This action set can be used to automate and end-to-end workflow or to automate steps in the  workflow such as data preparation, feature preprocessing, feature engineering, feature selection, and hyperparameter tuning.  In this notebook, we demonstrated how to use each step of the dataSciencePilot Action set from an R programmer's interface. 

In [33]:
cas.close(server)

[1] 0